# Frame extraction from Json files.

In [31]:
import json
from tqdm import tqdm
import pandas as pd
import cv2
import os
import numpy as np 

## Specify the file name

In [52]:
# Specify the path to the JSON file
file_path = 'GH020531.json'
with open(file_path, 'r') as file:
    data = json.load(file)

## Extract cyclist_id and frame_id from the json file

- the json is divided in frame and each frame contains object ids fro the n of cyclist dectected in the frame. 
- The following lines of code go thorugh the json file and extract the cyclist_id and frame_id from the json file. So that we know cyclist with object id "i" appears in which frames. With the frame number we also extract the bbox dimension.

In [53]:
obj_dict = {}
for i,frame  in tqdm(enumerate(data["detection"])):
    list_of_object = data["detection"][i]["objects"]

    for j,_ in enumerate(list_of_object): 
        if list_of_object[j]["obj_id"] in obj_dict.keys():
            if not [i, list_of_object[j]["bbox"]] in obj_dict[list_of_object[j]["obj_id"]]:
                obj_dict[list_of_object[j]["obj_id"]].append([i, list_of_object[j]["bbox"]])
        else:
            obj_dict[list_of_object[j]["obj_id"]] = [[i, list_of_object[j]["bbox"]]]

4499it [00:00, 873053.28it/s]


In [54]:
# Save the object dictionary as a JSON file
json.dump(obj_dict, open("object_ids.json", "w"), indent=4)

## Frame selection

here we select which frame is best for each cyclist detected. To do this we use the bbox dimension of the cyclist in the frame. We select the frame with the most centered frame box.

In [55]:
frame_to_extract = []
for i in obj_dict.keys():
    close_frame = float('inf')
    previous_distance= 1920
    for k,_ in enumerate(obj_dict[i]):
    
        mid_point = 0.5* ( obj_dict[i][k][1][0] + obj_dict[i][k][1][3] )
        if abs(650 - mid_point) < previous_distance:
            close_frame =  obj_dict[i][k][0]
            previous_distance = abs(650 - mid_point)
            
    frame_to_extract.append([i ,close_frame,obj_dict[i][k][1][0] + obj_dict[i][k][1][3]  ])      

In [56]:
frame_saved = pd.DataFrame(frame_to_extract, columns = ['object_id', 'frame_to_extract', "bbox_left", "bbox_right"])
frame_saved.sort_values(by = 'object_id', inplace = True)
frame_saved = frame_saved.reset_index(drop = True)
frame_saved.to_csv(file_path.split(".")[0] + "_frame_to_extract.csv", index = False)

Frame saved containes the object id (the ciclist) and the frame to extract.
We then use cv2 to extract the frame and saved them inthe folder. 

In [57]:
os.mkdir(file_path.split(".")[0])

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'GH020531'

In [ ]:
video = cv2.VideoCapture(file_path.split(".")[0] + ".MP4")
for j in frame_saved.frame_to_extract:   
    video.set(cv2.CAP_PROP_POS_FRAMES, j)
    ret, frame = video.read()
    
    # crop the frame based on bbox_left and bbox_right in frame_saved and add black patched on the cropped region
    #cropped_frame = frame[]

    # Create a black image of the same size as the cropped image
    #black_image = np.zeros(cropped_frame.shape, dtype=np.uint8)

    # Merge the black image with the cropped image
    #final_frame = cv2.addWeighted(cropped_frame, 0.7, black_image, 0.3, 0)
    
    cv2.imwrite( file_path.split(".")[0] + "/" + file_path.split(".")[0] + "_object_id_" + str(j) + ".png", frame)